1. 상관 분석 (Correlation Analysis)

In [1]:
import pandas as pd
bank = pd.read_csv('../bank.csv',sep=';')
print(bank.head())

   age          job  marital  education default  balance housing loan  \
0   30   unemployed  married    primary      no     1787      no   no   
1   33     services  married  secondary      no     4789     yes  yes   
2   35   management   single   tertiary      no     1350     yes   no   
3   30   management  married   tertiary      no     1476     yes  yes   
4   59  blue-collar  married  secondary      no        0     yes   no   

    contact  day month  duration  campaign  pdays  previous poutcome   y  
0  cellular   19   oct        79         1     -1         0  unknown  no  
1  cellular   11   may       220         1    339         4  failure  no  
2  cellular   16   apr       185         1    330         1  failure  no  
3   unknown    3   jun       199         4     -1         0  unknown  no  
4   unknown    5   may       226         1     -1         0  unknown  no  


In [2]:
#1. pdays와 balance의 공분산
covar = bank['pdays'].cov(bank['balance'])
print(covar)

2843.538052901605


In [3]:
#2. pday와 balance의 피어슨, p-val
from scipy.stats import pearsonr
pear,p_val = pearsonr(bank['pdays'],bank['balance'])
print('pearson coef',pear)
print('pval',p_val) #연관이 없음을 확인함

pearson coef 0.009436675990853845
pval 0.5258553742278886


In [4]:
#3 아래 코드를 완성하시고 결과를 출력하세요
# 상관분석2 - 스피어만 상관계수 - 순서 정보가 있는 ordinal한 범주형 변수간의 관계 
from scipy.stats import spearmanr
import numpy as np
import pandas as pd

X = [1, 2, 3, 4, 5]
Y = [5, 4, 3, 2, 1]

# 스피어만 상관계수 계산
rho, p_value = spearmanr(X, Y) # 적합한 변수를 넣으세요.

print("스피어만 상관계수:", rho)
print("p-value:", f"{p_value:.15f}")

alpha = 0.05

if p_value < alpha:
    print("귀무가설 기각. \n통계적으로 유의미한 수준으로 연관성이 있음.")
else:
    print("귀무가설 기각할 수 없음. \n통계적으로 유의미한 수준으로 연관성이 없음.")

스피어만 상관계수: -0.9999999999999999
p-value: 0.000000000000000
귀무가설 기각. 
통계적으로 유의미한 수준으로 연관성이 있음.


In [5]:
# 새로운 데이터
bicycle = pd.read_csv('../nyc-east-river-bicycle-counts.csv',index_col=0) #파일 불러올 때 index_col = 0 을 사용해서 첫번째 컬럼을 인덱스로 사용해버릴 수 있다.
print(bicycle.head())

                  Date                  Day  High Temp (°F)  Low Temp (°F)  \
0  2016-04-01 00:00:00  2016-04-01 00:00:00            78.1           66.0   
1  2016-04-02 00:00:00  2016-04-02 00:00:00            55.0           48.9   
2  2016-04-03 00:00:00  2016-04-03 00:00:00            39.9           34.0   
3  2016-04-04 00:00:00  2016-04-04 00:00:00            44.1           33.1   
4  2016-04-05 00:00:00  2016-04-05 00:00:00            42.1           26.1   

  Precipitation  Brooklyn Bridge  Manhattan Bridge  Williamsburg Bridge  \
0          0.01           1704.0              3126               4115.0   
1          0.15            827.0              1646               2565.0   
2          0.09            526.0              1232               1695.0   
3      0.47 (S)            521.0              1067               1440.0   
4             0           1416.0              2617               3081.0   

   Queensboro Bridge  Total  
0             2552.0  11497  
1             1884.0

2. 단순 선형 회귀(Simple Linear Regression)
- 종속 변수 Brooklyn Bridge, 독립변수 High Temp의 선형회귀모델을 구현하고 요약리포트 사용, 요약테이블 출력, 결과를 해석한 코멘트 3줄

In [6]:
X = bicycle['High Temp (°F)']
y = bicycle['Brooklyn Bridge']
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error, mean_squared_error

X = sm.add_constant(X) #상수 더해줌

model = sm.OLS(y,X).fit() #모델 선언후 바로 fit

pred = model.predict(X)

r_squared = model.rsquared
print(r_squared)
mae = mean_absolute_error(y,pred)
print('mae',mae)
rmse = np.sqrt(mean_squared_error(y,pred))
print('rmse',rmse)

summary = model.summary()

p_val_table = model.summary().tables[1]

print(summary)
print(p_val_table)

0.5466776174435135
mae 560.478806694349
rmse 659.0848680766909
                            OLS Regression Results                            
Dep. Variable:        Brooklyn Bridge   R-squared:                       0.547
Model:                            OLS   Adj. R-squared:                  0.544
Method:                 Least Squares   F-statistic:                     250.8
Date:                Wed, 07 Feb 2024   Prob (F-statistic):           1.37e-37
Time:                        13:21:11   Log-Likelihood:                -1661.1
No. Observations:                 210   AIC:                             3326.
Df Residuals:                     208   BIC:                             3333.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------

수치 및 리포트 결과 해석
1. r-squred의 값이 약 54%이상으로 절반정도 설명할 수 있다. 
2. coeffience의 방향성이 양의 영향관계를 띈다.
3. p-value의 값이 0에 수렴하기 때문에 영향을 준다고 볼 수 있다.

3. 다중 선형 회귀 모델 구현     
- 종속변수 brookly bridge, 독립변수 hightemp lowtemp 위와 동일함.

In [8]:
new_X = bicycle.copy()
col_list = ['High Temp (°F)','Low Temp (°F)','Brooklyn Bridge']
for i in new_X:
    if i not in col_list:
        new_X = new_X.drop(i,axis=1)
    else:
        pass

print(new_X.head())

y = new_X['Brooklyn Bridge']
new_X = new_X.drop('Brooklyn Bridge',axis=1)
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
new_X = scale.fit_transform(new_X)
new_X = sm.add_constant(new_X)
model = sm.OLS(y,new_X).fit()

pred = model.predict(new_X)

mae = mean_absolute_error(y,pred)
rmse = np.sqrt(mean_squared_error(y,pred))

print(model.summary())
print(model.summary().tables[1])

   High Temp (°F)  Low Temp (°F)  Brooklyn Bridge
0            78.1           66.0           1704.0
1            55.0           48.9            827.0
2            39.9           34.0            526.0
3            44.1           33.1            521.0
4            42.1           26.1           1416.0
                            OLS Regression Results                            
Dep. Variable:        Brooklyn Bridge   R-squared:                       0.612
Model:                            OLS   Adj. R-squared:                  0.608
Method:                 Least Squares   F-statistic:                     163.1
Date:                Wed, 07 Feb 2024   Prob (F-statistic):           2.97e-43
Time:                        13:42:31   Log-Likelihood:                -1644.8
No. Observations:                 210   AIC:                             3296.
Df Residuals:                     207   BIC:                             3306.
Df Model:                           2                               

1. R squared의 값은 0.612로 이 두가지 변수로 brooklyn bridge를 약 61.2% 설명 가능하다.
2. 두 변수 다 p-value의 값이 적으므로 연관이 있다고 볼 수 있다.
3. High temp는 뚜렷한 양의 영향관계를, Low Temp는 음의 영향관계를 가진다.

Logistic Regression(bank)

In [19]:
bank_data = pd.read_csv('../bank.csv',sep=';')
print(bank_data.head())
X = bank_data.select_dtypes(exclude='object').drop(['day'],axis=1)
y = bank_data['y'].apply(lambda x: 0 if x =='no' else 1)
col_list = list(X.columns)
col_list.insert(0,'const')
scale_X = StandardScaler().fit_transform(X)
X = sm.add_constant(scale_X)

model = sm.Logit(y,X).fit()
pred = model.predict()

   age          job  marital  education default  balance housing loan  \
0   30   unemployed  married    primary      no     1787      no   no   
1   33     services  married  secondary      no     4789     yes  yes   
2   35   management   single   tertiary      no     1350     yes   no   
3   30   management  married   tertiary      no     1476     yes  yes   
4   59  blue-collar  married  secondary      no        0     yes   no   

    contact  day month  duration  campaign  pdays  previous poutcome   y  
0  cellular   19   oct        79         1     -1         0  unknown  no  
1  cellular   11   may       220         1    339         4  failure  no  
2  cellular   16   apr       185         1    330         1  failure  no  
3   unknown    3   jun       199         4     -1         0  unknown  no  
4   unknown    5   may       226         1     -1         0  unknown  no  
Optimization terminated successfully.
         Current function value: 0.289092
         Iterations 7


학습 점검 체크리스트
- 공분산을 설명할 수 있는가? -> 두 변수간 변화하는 정도
- 피어슨 상관계수와 스피어만 상관계수의 차이를 설명할 수 있는가? -> 연속 / 범주형
- 피어슨 상관계수를 코드로 구현할 수 있는가? -> yes
- 데이터의 선형성을 설명할 수 있는가? -> yes
- 두 변수의 관계를 설명하는 상관계수를 그릴 수 있는가? -> yes
- 회귀계수와 상관계수의 차이를 설명할 수 있고 회귀 계수 결과를 설명할 수 있는가? -> 상관계수 : 선형관계의 강도 방향 / 회귀계수 : 변수의 영향관계
- 단순선형회귀와 다중선형회귀 결과 요약 리포트와 p-value 테이블을 출력할 수 있는가? -> yes
- 로지스틱 회귀분석은 이진 분류 문제에 사용되는 분석 방법이다.

In [24]:
mae = mean_absolute_error(y,pred)
summary = model.summary(xname=col_list)
print(summary)
print(summary.tables[1])
print(mae)


                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                 4521
Model:                          Logit   Df Residuals:                     4514
Method:                           MLE   Df Model:                            6
Date:                Wed, 07 Feb 2024   Pseudo R-squ.:                  0.1910
Time:                        14:12:10   Log-Likelihood:                -1307.0
converged:                       True   LL-Null:                       -1615.5
Covariance Type:            nonrobust   LLR p-value:                4.947e-130
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.4134      0.060    -40.174      0.000      -2.531      -2.296
age            0.1427      0.049      2.887      0.004       0.046       0.240
balance        0.0741      0.045      1.663      0.0

1. balance는 종속변수 y에 큰 영향력을 가지지 못한다, p-value 또한 높아서 balance를 제외하고 확인해도 무관하다.
2. duration이 영향관계가 가장 크다는 것을 알 수 있다. 
3. campaign을 제외하고 대부분 변수는 양의 영황관계를 가지고 있다. 
4. mae의 결과를 통해서 회귀선이 잘 예측하고 있음을 나타낸다.